<center><h1><b>NYC Apartment Search</b></h1></center>

---
# Setup

## Import Statements

This code block includes various import statements that bring in libraries and modules necessary for data manipulation, visualization, database operations, and geographic information system (GIS) tasks. These imports cover a wide range of functionalities required for the project.



In [2]:
# Standard library imports
import os
import json
import unittest
import pathlib
import urllib.parse
from datetime import datetime
from pathlib import Path
from typing import List

# Data manipulation and visualization
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import requests
import logging

# Database and GIS related imports
import psycopg2
import psycopg2.extras
import geoalchemy2 as gdb
import geopandas as gpd
import shapely
from shapely.geometry import Point, Polygon, mapping
from shapely import wkb

# SQLAlchemy imports
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import (
    create_engine,
    Column,
    Integer,
    Float,
    String,
    DateTime,
    text,
    ForeignKey,
)
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.schema import CreateTable
from geoalchemy2 import Geometry
from geoalchemy2.shape import to_shape


## Constants and File Paths

This code block defines several constants and file paths for data directories and files related to the project. It also includes constants for accessing New York City data through an API.


In [4]:
# File paths and directory constants
DATA_DIR = pathlib.Path("data")
ZIPCODE_DATA_FILE = DATA_DIR / "zipcodes" / "nyc_zipcodes.shp"
ZILLOW_DATA_FILE = DATA_DIR / "zillow_rent_data.csv"

# New York City data API constants
NYC_DATA_APP_TOKEN = "SMg9akfNT3gV1L4QAEb8vlx4F"
BASE_NYC_DATA_URL = "https://data.cityofnewyork.us/"
NYC_DATA_311 = "erm2-nwe9.geojson"
NYC_DATA_TREES = "5rq2-4hqu.geojson"

In [5]:
# Database constants
DB_NAME = "group19project"
DB_USER = "cecilialin"
DB_URL = f"postgres+psycopg2://{DB_USER}@localhost/{DB_NAME}"
DB_SCHEMA_FILE = "schema.sql"

In [6]:
# Directory for DB queries for Part 3
QUERY_DIR = pathlib.Path("queries")

# Make sure the QUERY_DIRECTORY exists
if not QUERY_DIR.exists():
    QUERY_DIR.mkdir()

In [7]:
# Directory path
directory = "data/resource"

# Create the directory. exist_ok=True means it won't throw an error if the directory already exists.
os.makedirs(directory, exist_ok=True)

print(f"Directory {directory} has been created.")


Directory data/resource has been created.


## Logging Setup

This code block sets up logging for information (INFO) and error tracking. It configures the logging module to handle log messages for the current module.


In [8]:
# Setup logging for info and error tracking
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)